In [1]:
import os
os.chdir("../expand_latex_macros/")
import source
import regex as re

In [2]:
latex_source_path = "../testing/1906.08350.tex"
str = source.expand_latex_macros(latex_source_path)
print(str)

Cannot handle recursively defined macro \miniscule. Not attempting.
\pdfoutput=1
\documentclass[12pt,a4paper]{article}
\usepackage{ifthen} \newboolean{pdflatex}
\setboolean{pdflatex}{true} 
\newboolean{articletitles}
\setboolean{articletitles}{true} 
\newboolean{uprightparticles}
\setboolean{uprightparticles}{false} 
\usepackage[top=1in, bottom=1.25in, left=1in, right=1in]{geometry}
\columnsep=5mm
\addtolength{\belowcaptionskip}{0.5em}
\renewcommand{\textfraction}{0.01}
\renewcommand{\floatpagefraction}{0.8} \renewcommand{\topfraction}{0.9}
\renewcommand{\bottomfraction}{0.9}
\raggedbottom
\sloppy
\usepackage{microtype}
\usepackage{lineno}  \usepackage{xspace} \usepackage{caption} \renewcommand{\captionfont}{\small}
\renewcommand{\captionlabelfont}{\small}
\usepackage{graphicx}  \usepackage{color}
\usepackage{colortbl}
\graphicspath{{./figs/}} \DeclareGraphicsExtensions{.pdf,.PDF,png,.PNG}
\usepackage{amsmath} \usepackage{amssymb}
\usepackage{amsfonts}
\usepackage{upgreek} 
\AtBeginDoc

In [3]:
latex_source = open(latex_source_path).read()
print(latex_source)

\pdfoutput=1


\documentclass[12pt,a4paper]{article}


\usepackage{ifthen} \newboolean{pdflatex}
\setboolean{pdflatex}{true} 

\newboolean{articletitles}
\setboolean{articletitles}{true} 

\newboolean{uprightparticles}
\setboolean{uprightparticles}{false} 



\def\paperauthors{LHCb collaboration} \def\paperasciititle{Precision measurement of the Lambdac, Xic and Xic0 baryon lifetimes} \def\papertitle{Precision measurement of the $\Lc$, $\Xicp$ and $\Xicz$ baryon lifetimes} \def\paperkeywords{{High Energy Physics}, {LHCb}} \def\papercopyright{\the\year\ CERN for the benefit of the LHCb collaboration} \def\paperlicence{CC-BY-4.0 licence}
\def\paperlicenceurl{https://creativecommons.org/licenses/by/4.0/}




\usepackage[top=1in, bottom=1.25in, left=1in, right=1in]{geometry}



\columnsep=5mm
\addtolength{\belowcaptionskip}{0.5em}

\renewcommand{\textfraction}{0.01}
\renewcommand{\floatpagefraction}{0.8} \renewcommand{\topfraction}{0.9}
\renewcommand{\bottomfraction}{0.9}

\raggedbottom
\s

In [7]:
code = """
\\def\\mean#1#2 {(#1 + #2) / 2}
\\newcommand{\\topmean}[3]{\\mean {2} {#1} top #2 #3 top \\mean{#2}{#1}}
\\def\\musymbol {\\mu}
\\def\\musymbolargs#1#2 {\\mu_{#1}^{#2}}
\\newcommand{\\thiscommand}[2]{\\topmean{#1}{#1}{#1} \\musymbol \\musymbolargs{1}{2}}
"""

In [8]:
commands = source.parse_macros(code)
commands

{'\\mean': '(#1 + #2) / 2',
 '\\musymbol': '\\mu',
 '\\musymbolargs': '\\mu_{#1}^{#2}',
 '\\topmean': '\\mean {2} {#1} top #2 #3 top \\mean{#2}{#1}',
 '\\thiscommand': '\\topmean{#1}{#1}{#1} \\musymbol \\musymbolargs{1}{2}'}

In [9]:
source.expand_nested_macros(commands)

{'\\mean': '(#1 + #2) / 2',
 '\\musymbol': '\\mu',
 '\\musymbolargs': '\\mu_{#1}^{#2}',
 '\\topmean': '(2 + #1) / 2 top #2 #3 top (#2 + #1) / 2',
 '\\thiscommand': '(2 + #1) / 2 top #1 #1 top (#1 + #1) / 2 \\mu \\mu_{1}^{2}'}

In [130]:
def sub_command_for_def(string, command, definition):
    # Check if command definition uses args
    # If yes args
    if re.search(r"#\d+", definition):
        pattern = re.escape(command) + r"((?:\s*\{[^}]+\})+)"
        args = re.findall(pattern, string)
        pattern = r"\{([^}]+)\}"
        expanded_args = []
        for arg in args:
            expanded_args.append(re.findall(pattern, arg))
        print(args, expanded_args)
        for i, arg in enumerate(args):
            command_call = command + arg
            sub_for_args = {}
            for j, expanded_arg in enumerate(expanded_args[i]):
                sub_for_args[f"#{j+1}"] = expanded_arg
            pattern = re.compile("|".join(re.escape(key) for key in sub_for_args.keys()))
            subbed_definition = pattern.sub(lambda match: sub_for_args[match.group(0)], definition)
            string = string.replace(command_call, subbed_definition)
        return string
    # If no args
    else:
        pattern = re.escape(command) + r"\b"
        definition = definition.replace('\\', '\\\\')
        return re.sub(pattern, definition, string)

string = commands['\\thiscommand']
command = "\\musymbol"
definition = commands[command]
sub_command_for_def(string, command, definition)

'\\topmean{#1}{#1}{#1} \\mu \\musymbolargs{#1}{#1}'

In [ ]:

command_def = commands[command]
for i, arg in enumerate(args):
    command_def = command_def.replace(f'#{i}', arg)
print(command_def)

In [54]:
commands["\\topmean"]

'\\mean {2 } {#1 } top #2 #3'

In [10]:
source.expand_nested_macros(commands)

{'\\mean': 'mean of #1 #2', '\\topmean': 'mean of #1 #2{#1}{#2} top #3'}